In [17]:
import os
import io
import sys
import time
import pymp
import h5py
import urllib 
import ctypes
import pymorton as pym
import requests
import numpy as np
import pyJHTDB
from pyJHTDB.dbinfo import isotropic1024coarse
from pyJHTDB import libJHTDB

In [18]:
os.environ['TMPDIR']='/home/idies/workspace/scratch'

Nx = isotropic1024coarse['nx']; Ny = isotropic1024coarse['ny']; Nz = isotropic1024coarse['nz']
Lx = isotropic1024coarse['lx']; Ly = isotropic1024coarse['ly']; Lz = isotropic1024coarse['lz']

dataset = 'isotropic1024coarse'
getFunction='Velocity'
t = 0.0; nx=Nx; ny=Ny; nz=Nz
chkSz = 32; slabs = nx//chkSz

In [21]:
# listing in HTML
url='http://dsp033.pha.jhu.edu:8000/turbulence/'
data=urllib.urlopen(url).read()

print("%s : \nsize of data [bytes] : %d"%(url,len(data)))

http://dsp033.pha.jhu.edu:8000/turbulence/ : 
size of data [bytes] : 104306


In [22]:
t1 = time.time()

idx = 0
dsp033_url = 'http://dsp033.pha.jhu.edu:8000/turbulence/turbdb10'+str(idx+1)+'_0.bin'

d=urllib.urlopen(dsp033_url).read()
n=int(len(d)/4)
_arr=np.frombuffer(d,dtype=np.float32,count=n)
d=None
        
t2 = time.time()
sys.stdout.write('Getting the data: {0:.2f} seconds\n'.format(t2-t1))

Getting the data: 23.71 seconds


In [16]:
t1 = time.time()

threads = 8

with pymp.Parallel(threads) as p:
    for idx in p.range(0,threads):
        #virgo_url = 'http://dsp060/disk0'+str(idx+1)+'/filedb/turbulence/turbdb10'+str(idx+1)+'_0.bin'
        #virgo_url = 'http://dsp033.pha.jhu.edu:8080/turbdb10'+str(idx+1)+'_0.bin'
        #dsp033_url = 'http://dsp033.pha.jhu.edu:8080/turbulence/turbdb10'+str(idx+1)+'_0.bin'
        dsp033_url = 'http://dsp033.pha.jhu.edu:8000/turbulence/turbdb10'+str(idx+1)+'_0.bin'
        
        print(dsp033_url)
        #d=urllib.urlopen(virgo_url).read()
        d=urllib.urlopen(dsp033_url).read()
        
        n=int(len(d)/4)
        print(idx,len(d)/4)
        _arr=np.frombuffer(d,dtype=np.float32,count=n)
        d=None
        
t2 = time.time()
sys.stdout.write('Getting the data: {0:.2f} seconds\n'.format(t2-t1))

http://dsp033.pha.jhu.edu:8000/turbulence/turbdb102_0.bin
http://dsp033.pha.jhu.edu:8000/turbulence/turbdb107_0.bin
http://dsp033.pha.jhu.edu:8000/turbulence/turbdb105_0.bin
http://dsp033.pha.jhu.edu:8000/turbulence/turbdb106_0.bin
http://dsp033.pha.jhu.edu:8000/turbulence/turbdb103_0.bin
http://dsp033.pha.jhu.edu:8000/turbulence/turbdb104_0.bin
http://dsp033.pha.jhu.edu:8000/turbulence/turbdb108_0.bin
http://dsp033.pha.jhu.edu:8000/turbulence/turbdb101_0.bin
(1, 402653184)


IOError: [Errno socket error] [Errno 104] Connection reset by peer

In [4]:
t1 = time.time()

zmin = pym.interleave3(0,0,0)
zmax = pym.interleave3(511,511,511)
bs = 8
blobsize = bs*bs*bs

nb = 512
uBlock = np.zeros((nb,nb,nb),dtype='float32')
vBlock = np.zeros((nb,nb,nb),dtype='float32')
wBlock = np.zeros((nb,nb,nb),dtype='float32')

for zindex in range(zmin,zmax,blobsize):
    coord = pym.deinterleave3(zindex)
    c0 = coord[0]; c1 = coord[1]; c2 = coord[2]
        
    blob = _arr[3*zindex:3*(zindex+blobsize)]
    block = np.reshape(blob,(bs,bs,bs,3),order='C')
    
    uBlock[c0:(c0+bs),c1:(c1+bs),c2:(c2+bs)] = np.transpose(block[:,:,:,0])
    vBlock[c0:(c0+bs),c1:(c1+bs),c2:(c2+bs)] = np.transpose(block[:,:,:,1])
    wBlock[c0:(c0+bs),c1:(c1+bs),c2:(c2+bs)] = np.transpose(block[:,:,:,2])
    
t2 = time.time()
sys.stdout.write('Reshaping: {0:.2f} seconds\n'.format(t2-t1))

Reshaping: 10.49 seconds


In [5]:
t1 = time.time()

shu = pymp.shared.array((Nx,Ny,Nz), dtype='float32')
shv = pymp.shared.array((Nx,Ny,Nz), dtype='float32')
shw = pymp.shared.array((Nx,Ny,Nz), dtype='float32')

t2 = time.time()
sys.stdout.write('Alocating the shared arrays: {0:.2f} seconds\n'.format(t2-t1))

Alocating the shared arrays: 20.32 seconds


In [16]:
t1 = time.time()

threads = 8

zmin = pym.interleave3(0,0,0)
zmax = pym.interleave3(511,511,511)+1
bs = 8
blobsize = bs*bs*bs

with pymp.Parallel(threads) as p:
    for idx in p.range(0,threads):
        if idx==0:
            print("Downloading")
        
        virgo_url = 'http://dsp060/disk0'+str(idx+1)+'/filedb/turbulence/turbdb10'+str(idx+1)+'_0.bin'
        d=urllib.urlopen(virgo_url).read()
        n=int(len(d)/4)
        _arr=np.frombuffer(d,dtype=np.float32,count=n)
        d=None
        
        ######################################
        
        if idx==0:
            print("z-reordering")
            
        nb = 512
        uBlock = np.zeros((nb,nb,nb),dtype='float32')
        vBlock = np.zeros((nb,nb,nb),dtype='float32')
        wBlock = np.zeros((nb,nb,nb),dtype='float32')
        
        for zindex in range(zmin,zmax,blobsize):
            coord = pym.deinterleave3(zindex)
            c0 = coord[0]; c1 = coord[1]; c2 = coord[2]
            
            blob = _arr[3*zindex:3*(zindex+blobsize)]
            block = np.reshape(blob,(bs,bs,bs,3),order='C')
            
            uBlock[c0:(c0+bs),c1:(c1+bs),c2:(c2+bs)] = np.transpose(block[:,:,:,0])
            vBlock[c0:(c0+bs),c1:(c1+bs),c2:(c2+bs)] = np.transpose(block[:,:,:,1])
            wBlock[c0:(c0+bs),c1:(c1+bs),c2:(c2+bs)] = np.transpose(block[:,:,:,2])
            
        #######################################
        
        if idx==0:
            print("writing the blocks on the right places")
                
        Coord = pym.deinterleave3(idx)
        C0 = Coord[0]; C1 = Coord[1]; C2 = Coord[2]
        
        shu[C0*nb:(C0+1)*nb,C1*nb:(C1+1)*nb,C2*nb:(C2+1)*nb] = uBlock[:,:,:]
        shv[C0*nb:(C0+1)*nb,C1*nb:(C1+1)*nb,C2*nb:(C2+1)*nb] = vBlock[:,:,:]
        shw[C0*nb:(C0+1)*nb,C1*nb:(C1+1)*nb,C2*nb:(C2+1)*nb] = wBlock[:,:,:]
        
        ########################################
            
        del uBlock,vBlock,wBlock
        
t2 = time.time()
sys.stdout.write('Getting the data: {0:.2f} seconds\n'.format(t2-t1))

Downloading
z-reordering
writing the blocks on the right places
Getting the data: 39.40 seconds


In [7]:
t1 = time.time()

folder = "/home/idies/workspace/scratch"
h5f = h5py.File(folder+'/'+'velocity-zfileDB.h5','w')
h5f.create_dataset('u',data=shu)
h5f.create_dataset('v',data=shv)
h5f.create_dataset('w',data=shw)
h5f.close()

t2 = time.time()
sys.stdout.write('Write hdf5 file: {0:.2f} seconds\n'.format(t2-t1))

Write hdf5 file: 28.47 seconds


In [8]:
t1 = time.time()
nproc = 8
for k in range(nproc):
    folder = "/home/idies/workspace/scratch"
    filename = "zturbdb-isotropic1024coarse-"+str(k)+".npz"
    filet = folder + "/" + filename
    np.savez(filet,u=shu[k*(Nx//nproc):(k+1)*(Nx//nproc),:,:],v=shv[k*(Nx//nproc):(k+1)*(Nx//nproc),:,:],w=shw[k*(Nx//nproc):(k+1)*(Nx//nproc),:,:],nproc=nproc)
t2 = time.time()
sys.stdout.write('Write in disk: {0:.2f} seconds\n'.format(t2-t1))

Write in disk: 145.80 seconds
